# 2.5D DC Resistivity Forward Simulation

This tutorial demonstrates forward simulation of DC resistivity data with SimPEG. We define surface topography, survey geometry and subsurface electrical resistivity for a 2.5D geometry. We use this information to simulate normalized voltage data.

The following items are dicussed in this tutorial:

- Defining surface topography.
- Defining surveys within SimPEG.
- Defining the mesh on which the forward simulation is solved.
- Defining a model which represents Earth's subsurface resistivity.
- Defining the forward simulation.
- Plotting DC resistivity data.

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/dcr_fwd.png" width="100%" align="center"/>

## Step 0: Importing Modules

In [ ]:
# SimPEG functionality
from SimPEG.electromagnetics.static import resistivity as dc
from SimPEG.utils import model_builder
from SimPEG import maps
from SimPEG.electromagnetics.static.utils.static_utils import (
    generate_dcip_sources_line,
    pseudo_locations,
    plot_pseudosection,
    apparent_resistivity_from_voltage,
)

# discretize functionality
from discretize import TreeMesh
from discretize.utils import active_from_xyz

# Common Python functionality
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

mpl.rcParams.update({"font.size": 14})  # default font size

write_output = True  # Optional

## Step 1: Defining Topography

<img src="https://www.researchgate.net/profile/Oyebanji-Alagbo-2/publication/359861220/figure/fig1/AS:1143365911478274@1649611084348/Schematic-description-of-ridge-tillage-with-crops-placed-on-top-of-the-re-compacted.png" width="75%" align="center"/>

### <span style="color:darkgreen">Exercise (beginner):</span>

Define a topography over x = (-300, 300) with a 1 m horizontal spacing. Let the elevation $y = 4 + 4 \, tanh(x/100)$. Use the variable name *topo_2d* to define the (N, 2) numpy array for the topography.

In [ ]:
fig = plt.figure(figsize=(6, 2))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
ax.plot(topo_2d[:, 0], topo_2d[:, -1], color="b", linewidth=2)
ax.set_xlabel("x (m)", labelpad=5)
ax.set_ylabel("y (m)", labelpad=5)
ax.grid(True)
ax.set_title("Topography", fontsize=16, pad=10)
plt.show(fig)

## Step 2: Defining the Survey

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/DCR_DipoleDipole_Array.svg" width="40%" align="right"/>

DC resistivity surveys within SimPEG require the user to create and connect three types of objects:

- receivers
- sources
- survey

These can be defined manually, or efficiently using SimPEG utilities.


### <span style="color:darkgreen">Exercise (beginner):</span>

Use the [generate_dcip_sources_line](myst:SimPEG#SimPEG.electromagnetics.static.utils.generate_dcip_sources_line) utility function to define a DC resistivity survey line. Enter the following input arguments, in order:

* **'dipole-dipole'** (survey type)
* **'volt'** (data type)
* **'2D'** (dimension of the problem)
* **[-65, 65]** (horizontal end locations)
* **topo_2d** (our topography object)
* **15** (number of receivers per source)
* **5** (electrode spacing)

This will output a list of source objects with associated receivers. Then define a [dc.Survey](https://docs.simpeg.xyz/content/api/SimPEG.electromagnetics.static.resistivity.html#survey) the object which stores and organizes all of the sources and receivers.

### <span style="color:darkorange">Exercise (advanced):</span>

Try different electrode configurations in your survey (e.g. 'pole-dipole'). Or change the number of receivers or the electrode spacing.

In [ ]:
pseudo_locations_xy = pseudo_locations(survey)
fig = plt.figure(figsize=(8, 2.75))
ax = fig.add_axes([0.1, 0.1, 0.85, 0.8])
ax.scatter(pseudo_locations_xy[:, 0], pseudo_locations_xy[:, -1], 8, "r")
ax.set_xlabel("x (m)")
ax.set_ylabel("y (m)")
ax.set_title("Pseudo-locations")
plt.show()

### <span style="color:darkgreen">Exercise (beginner):</span>

Extract the following items from the survey object:

- the total number of data
- the electrode locations
- the 3rd receiver of the 2nd source

## Step 3: Designing a Mesh

Meshes define the numerical grid on which we numerically solve the PDE for the DC resistivity problem. Here, we define a tree mesh.

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/sphx_glr_1_mesh_overview_001.png" width="75%" align="center"/>

### <span style="color:darkgreen">Exercise (beginner):</span>

Define and provide reasonable values for the following variables:

* **dh** (minimum cell width)
* **dom_width_x** (width of the domain along x)
* **dom_width_y** (width of the domain along y)

### <span style="color:darkorange">Exercise (advanced):</span>

Play with the parameters of the [refine_surface](https://discretize.simpeg.xyz/en/main/api/generated/discretize.TreeMesh.refine_surface.html) and [refine_points](https://discretize.simpeg.xyz/en/main/api/generated/discretize.TreeMesh.refine_points.html) methods to change the level of discretization around the surface and the electrode locations. Or visit the [tree mesh](https://discretize.simpeg.xyz/en/main/api/generated/discretize.TreeMesh.html) API to see additional refinement methods.

In [ ]:
# Number of base cells along x and y
nbcx = 2 ** int(np.round(np.log(dom_width_x / dh) / np.log(2.0)))
nbcy = 2 ** int(np.round(np.log(dom_width_y / dh) / np.log(2.0)))

# Define the base mesh with top at z = 0 m.
hx = [(dh, nbcx)]
hy = [(dh, nbcy)]
mesh = TreeMesh([hx, hy], origin="CN")

# Shift top to maximum topography and center of survey line
y_topo_max = np.max(topo_2d[:, -1])
mesh.origin = mesh.origin + np.r_[np.mean(end_locations), y_topo_max]

In [ ]:
# Mesh refinement near electrodes.
unique_locations = survey.unique_electrode_locations

mesh.refine_points(
    unique_locations, padding_cells_by_level=[10, 8, 8, 8, 4, 4], finalize=False
)

# Mesh refinement based on topography
mesh.refine_surface(
    topo_2d[np.abs(x_topo) < 150.0, :],
    padding_cells_by_level=[0, 0, 4, 4],
    finalize=False,
)

In [ ]:
# Finalize
mesh.finalize()

In [ ]:
fig = plt.figure(figsize=(8, 4))

ax1 = fig.add_axes([0.14, 0.17, 0.8, 0.7])
mesh.plot_grid(ax=ax1, linewidth=1)
ax1.grid(False)
ax1.set_xlim(-200, 200)
ax1.set_ylim(y_topo_max - 200, y_topo_max)
ax1.set_title("Mesh")
ax1.set_xlabel("x (m)")
ax1.set_ylabel("y (m)")

plt.show()

### <span style="color:darkgreen">Exercise (beginner):</span>

The mesh is another object whose properties we can extract. Try to exctract the following properties from the mesh:

- the total number of cells
- the origin
- the cell centers

## Step 4: Active Cells

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/active_cells.png" width="25%" align="right"/>

### <span style="color:darkgreen">Exercise (beginner):</span>

Use the [active_from_xyz](https://discretize.simpeg.xyz/en/main/api/generated/discretize.utils.active_from_xyz.html) utility function define the active mesh cells. The input arguments for the function are the

* **mesh** (the tree mesh)
* **topo_2d** (2D topography)

The output quantity is a ``bool`` array which should be named **active_cells**.

## Step 5: Defining a Model

### <span style="color:darkgreen">Exercise (beginner):</span>

Generate a model whose parameters define the resistivities ($\Omega m$) on all active mesh cells. The model will have the following features:

* Near surface resistivity of 100 $\Omega m$
* Basement resistivity (z < -16 m) of 10 $\Omega m$
* An infinite cylinder centered at (10, -5) with a radius of 5 m with a resistivity of 1000 $\Omega m$

*For simple geometric shapes, we can use utilities like* [model_builder.get_indices_sphere](https://docs.simpeg.xyz/content/api/generated/SimPEG.utils.model_builder.get_indices_sphere.html).

### <span style="color:darkorange">Exercise (advanced):</span>

Generate a model whose parameters define the log-resistivities (natural log) on all active mesh cells.

In [ ]:
plotting_map = maps.InjectActiveCells(mesh, active_cells, np.nan)

fig = plt.figure(figsize=(10, 3))

norm = LogNorm(vmin=1e1, vmax=1e3)

ax1 = fig.add_axes([0.14, 0.17, 0.68, 0.7])
mesh.plot_image(
    plotting_map * resistivity_model,
    ax=ax1,
    grid=False,
    pcolor_opts={"norm": norm, "cmap": mpl.cm.RdYlBu_r},
)
ax1.set_xlim(-70, 70)
ax1.set_ylim(y_topo_max - 50, y_topo_max)
ax1.set_title("Electrical Resistivity (Active Cells)")
ax1.set_xlabel("x (m)")
ax1.set_ylabel("y (m)")

ax2 = fig.add_axes([0.84, 0.17, 0.03, 0.7])
cbar = mpl.colorbar.ColorbarBase(
    ax2, norm=norm, orientation="vertical", cmap=mpl.cm.RdYlBu_r
)
cbar.set_label(r"$\rho (\Omega m)$", rotation=270, labelpad=25, size=16)

plt.show()

## Step 6: Mapping from the Model to the Mesh

### <span style="color:darkgreen">Exercise (beginner):</span>

Define a model representing electrical resistivities on all active cells. Use the [maps.InjectActiveCells](myst:SimPEG#SimPEG.maps.InjectActiveCells) mapping to define this mapping. **Important:** although the true electrical resistivity of the air is infinity, set the value for the inactive cells to 1e8 $\Omega m$!

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/mapping_1.png" width="50%" align="center"/>

### <span style="color:darkorange">Exercise (advanced):</span>

Define a model representing log-resistivities on all active cells. Our mapping must: 

- converts log-resistivities to resistivities, then
- projects these values to the entire mesh AND sets a static value for all air cells.

Use the [maps.InjectActiveCells](https://docs.simpeg.xyz/content/api/generated/SimPEG.maps.InjectActiveCells.html#SimPEG.maps.InjectActiveCells) and [maps.ExpMap](https://docs.simpeg.xyz/content/api/generated/SimPEG.maps.ExpMap.html#SimPEG.maps.ExpMap) mappings to accomplish this. Successive mappings can be combined via the chain rule using the $^*$ operator; i.e.

```
rho_map = active_map * exp_map
```

## Step 7: Project Survey to Discretized Topography

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/project_electrodes.png" width="40%" align="center"/>

### <span style="color:darkgreen">Exercise (beginner):</span>

Use the [drape_electrodes_on_topography](https://docs.simpeg.xyz/content/api/generated/SimPEG.electromagnetics.static.resistivity.Survey.drape_electrodes_on_topography.html#SimPEG.electromagnetics.static.resistivity.Survey.drape_electrodes_on_topography) method to project surface electrodes to the discrete surface. The input arguments are:

* **mesh** (the tree mesh)
* **active_cells** (your active cells array)

This method will reassign the locations of the electrode in all source and receiver objects.

## Step 8: Defining the Forward Simulation

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/fwd_simulation.png" width="30%" align="right"/>

In SimPEG, the governing physics, problem geometry and input parameters for the forward simulation are defined within simulation classes. The simulation object must be associated with a:

- survey
- mesh
- mapping from the model to the mesh

all of which we created earlier. The numerical formulation representing the governing physics is specific to each simulation class.

### <span style="color:darkgreen">Exercise (beginner):</span>

Use [dc.simulation_2d.Simulation2DNodel](https://docs.simpeg.xyz/content/api/generated/SimPEG.electromagnetics.static.resistivity.Simulation2DNodal.html#SimPEG.electromagnetics.static.resistivity.Simulation2DNodal) to define the simulation. We instantiate the simulation by setting the following:

* **mesh** (the mesh as an input argument)
* **survey=survey** (set the survey as a keyword argument)
* **rhoMap=resistivity_map** (set the mapping as a keyword argument)

## Step 9: Predict DC Resistivity Data and Plot

### <span style="color:darkgreen">Exercise (beginner):</span>

Use the **dpred** method to simulate DC resistivity data for your resistivity model.

## Plot DC Resistivity Data

Here we use the [plot_pseudosection](https://docs.simpeg.xyz/content/api/generated/SimPEG.electromagnetics.static.utils.plot_pseudosection.html#SimPEG.electromagnetics.static.utils.plot_pseudosection) utility function to represent the predicted data on a pseudosection plot as apparent conductivities. If the receivers were defined to simulate data as normalized voltages, we may want to use the [apparent_resistivity_from_voltage](https://docs.simpeg.xyz/content/api/generated/SimPEG.electromagnetics.static.utils.apparent_resistivity_from_voltage.html#SimPEG.electromagnetics.static.utils.apparent_resistivity_from_voltage) utility function to convert the data to apparent resistivities.

In [ ]:
# Plot voltages pseudo-section
if data_type == "volt":
    fig = plt.figure(figsize=(8, 2.75))
    ax1 = fig.add_axes([0.1, 0.15, 0.75, 0.78])
    plot_pseudosection(
        survey,
        dobs=np.abs(dpred),
        plot_type="scatter",
        ax=ax1,
        scale="log",
        cbar_label="V/A",
        scatter_opts={"cmap": mpl.cm.viridis},
    )
    ax1.set_title("Normalized Voltages")
    plt.show()

    # Get apparent conductivities from volts and survey geometry
    apparent_resistivity = apparent_resistivity_from_voltage(survey, dpred)

else:
    apparent_resistivity = dpred.copy()

# Plot apparent resistivity pseudo-section
fig = plt.figure(figsize=(8, 2.75))
ax1 = fig.add_axes([0.1, 0.15, 0.75, 0.78])
plot_pseudosection(
    survey,
    dobs=apparent_resistivity,
    data_locations=True,
    plot_type="contourf",
    ax=ax1,
    scale="log",
    cbar_label="$\Omega m$",
    mask_topography=True,
    contourf_opts={"levels": 40, "cmap": mpl.cm.RdYlBu_r},
)
ax1.set_title("Apparent Resistivity")
plt.show()